In [1]:
import ccxt

binance = ccxt.binance()
current_pair = 'ETH/USDT'

# Get the current price of the pair
price = binance.fetch_ticker(current_pair)['last']
price

3846.39

In [2]:
import ccxt
import pandas as pd
import numpy as np
try:
    import talib
except ImportError:
    print("Please install talib: pip install TA-Lib")
    exit(1)

def get_btc_rsi(exchange_name='binance', symbol='BTC/USDT', timeframe='1h', period=14):
    """
    Calculate current RSI for BTC using ccxt and talib
    
    Parameters:
    - exchange_name: Exchange to use (default: 'binance')
    - symbol: Trading pair (default: 'BTC/USDT')
    - timeframe: Candle timeframe (default: '1h')
    - period: RSI period (default: 14)
    """
    
    # Initialize exchange
    exchange = getattr(ccxt, exchange_name)()
    
    # Fetch OHLCV data
    # We need at least 2*period candles for accurate RSI calculation
    limit = period * 3
    ohlcv = exchange.fetch_ohlcv(symbol, timeframe=timeframe, limit=limit)
    
    # Convert to pandas DataFrame
    df = pd.DataFrame(ohlcv, columns=['timestamp', 'open', 'high', 'low', 'close', 'volume'])
    df['timestamp'] = pd.to_datetime(df['timestamp'], unit='ms')
    
    # Calculate RSI using talib
    rsi = talib.RSI(df['close'].values, timeperiod=period)
    
    # Get the current RSI value (last non-NaN value)
    current_rsi = rsi[-1]
    
    # Get current price
    current_price = df['close'].iloc[-1]
    
    return {
        'symbol': symbol,
        'price': current_price,
        'rsi': current_rsi,
        'timeframe': timeframe,
        'period': period,
        'timestamp': df['timestamp'].iloc[-1]
    }

# Main execution
if __name__ == "__main__":
    try:
        # Get BTC RSI
        result = get_btc_rsi(timeframe='1m', period=14)
        
        print(f"Bitcoin ({result['symbol']}) Technical Analysis")
        print(f"{'='*40}")
        print(f"Current Price: ${result['price']:,.2f}")
        print(f"RSI ({result['period']}): {result['rsi']:.2f}")
        print(f"Timeframe: {result['timeframe']}")
        print(f"Timestamp: {result['timestamp']}")
        print(f"\nRSI Interpretation:")
        
        if result['rsi'] > 70:
            print("⚠️ OVERBOUGHT - RSI above 70")
        elif result['rsi'] < 30:
            print("⚠️ OVERSOLD - RSI below 30")
        else:
            print("✅ NEUTRAL - RSI between 30-70")
            
    except Exception as e:
        print(f"Error: {e}")

Bitcoin (BTC/USDT) Technical Analysis
Current Price: $119,041.30
RSI (14): 44.37
Timeframe: 1m
Timestamp: 2025-07-28 02:05:00

RSI Interpretation:
✅ NEUTRAL - RSI between 30-70
